In [1]:
import sqlite3
import datetime

alarms

In [7]:
alarm_db = 'database/alarmtimes.db' 

In [79]:
def request_handler(request):
    if request["method"]=="POST":
        #parse the body of the incoming POST
        try:
            username = request['form']['user']
            times = request['form']['alarm_time'][1:-1].replace(' ','').split(',')
            alarm_music = request['form']['music'][1:-1].split(',')
        except:
            return "Error with username, alarm time, set state, or music"   
        
        alarms = [t[1:-1] for t in times]
        musics = [int(m) for m in alarm_music]

        if len(alarms) != len(musics):
            return "Error with alarm and music matching"

        with sqlite3.connect(alarm_db) as c:
            c.cursor()  # move cursor into database (allows us to execute commands)
            c.execute("""CREATE TABLE IF NOT EXISTS alarm_data (user text, alarm_time text, music int);""")
            # some other query(ies) about inserting data
            
            for i in range(len(alarms)):
                print( alarms[i], musics[i])
                try:
                    c.execute('''DELETE FROM alarm_data WHERE user == ?''', (username,))
                except:
                    print('user not in database yet')
                c.execute('''INSERT into alarm_data VALUES (?,?,?);''',(username, alarms[i], musics[i]))

        return "Data POSTED successfully"
    else:
        return "invalid HTTP method for this url."

In [68]:
p = {'method': 'POST', 'form': {'user': 'bat', 'alarm_time': "['02:30', '12:00']", 'music': "[0,1]"}, 'args': ['user', 'alarm_time', 'music']}

In [80]:
request_handler(p)

['02:30', '12:00']
02:30 0
12:00 1


'Data POSTED successfully'

get alarms

In [20]:
def alarm_request_handler(request):
    if request["method"]=="GET":
        try:
            username = request['values']['user']
        except:
            return "This request is missing a username"   
        
        with sqlite3.connect(alarm_db) as c:
            
            #check if exists
            users = [i for i in c.execute('''SELECT * FROM alarm_data WHERE user == ?;''',(username,))]
            
            if not users:
                return "User does not exist or has no alarms set"

            times = [i[1] for i in users]
            music = [j[1] for j in users]
            
        return {'alarm_time': times, 'music': music}
    else:
      return "invalid HTTP method for this url."

In [18]:
g = {'method': 'GET', 'values': {'user': 'cat'}, 'args': ['user']}

In [21]:
alarm_request_handler(g)

{'alarm_time': ['02:30', '12:00'], 'music': ['02:30', '12:00']}

leaderboard

In [90]:
game_db = 'database/leaderboard.db' 

In [91]:
def post_request_handler(request):
    if request["method"]=="POST":
        #parse the body of the incoming POST
        try:
            username = request['form']['user']
            gamename = request['form']['game_name']
            gamelength = request['form']['length']
        except:
            return "Error with username, game name, or game length"   
        
        with sqlite3.connect(game_db) as c:
            c.cursor()  # move cursor into database (allows us to execute commands)
            c.execute("""CREATE TABLE IF NOT EXISTS all_game_data (user text, date Datetime, game_name text, length float);""")
            # some other query(ies) about inserting data
            c.execute('''INSERT into all_game_data VALUES (?,?,?,?);''',(username, datetime.datetime.now(), gamename, gamelength))

        return "Data POSTED successfully"
    else:
        return "invalid HTTP method for this url."

In [104]:
user_game = {'method': 'POST', 'form': {'user': 'cat', 'game_name': 'math', 'length': '13.5'}, 'args': ['user', 'game_name', 'length']}
#request_handler(user_game)

In [105]:
post_request_handler(user_game)

'Data POSTED successfully'

In [109]:
def get_request_handler(request):
    if request["method"]=="GET":
        #parse the body of the incoming POST
        try:
            username = request['values']['user']
            gamename = request['values']['game_name']
        except:
            return "Error with username or user has not played this game"   
        
        
        with sqlite3.connect(game_db) as c:
            c.cursor()  # move cursor into database (allows us to execute commands)
            users_games = [i for i in c.execute('''SELECT * FROM all_game_data WHERE user == ? AND game_name = ? ORDER BY length ASC;''',(username,gamename))]
            print(users_games)

            

        return "Data Gotten successfully"
    else:
        return "invalid HTTP method for this url."

In [107]:
ug = {'method': 'GET', 'values': {'user': 'cat', 'game_name': 'math'}, 'args': ['user', 'game_name']}

In [110]:
get_request_handler(ug)

[('cat', '2022-04-27 06:23:15.767295', 'math', 13.5), ('cat', '2022-04-27 06:23:18.865893', 'math', 13.5)]


'Data Gotten successfully'

bokeh

In [87]:
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, TableColumn, DataTable, CustomJS, Range1d, Select
from bokeh.io import show
from bokeh.layouts import column
from bokeh.embed import components
import pandas as pd
import sqlite3
import datetime
from bokeh.models import DatetimeTickFormatter

In [86]:
#game over time
from bokeh.models import ColumnDataSource, CustomJS, Range1d, Select
from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import column

import pandas as pd
import numpy as np

# you can also output to an HTML file
output_notebook() 

# toy dataframe with two y columns you can switch between
df = pd.DataFrame(data={
    "x"     : range(0,100),
    "y1"     : np.random.randint(10, 20, 100),
    "y2"     :  np.random.randint(40, 50, 100)
})

# fix the y_range to be the same for two lines
p = figure(y_range=Range1d(0, 60), plot_width=600, plot_height=400)

# shared datasource
source = ColumnDataSource(df)

plot_1 = p.line(x="x", y="y1", color="teal", source=source, line_width=1)
plot_2 = p.line(x="x", y="y2", color="firebrick", source=source, line_width=1)

# initialise the plot with only y1 visible - to match the dropdown default
plot_2.visible = False

# dropdown widget + Javascript code for interactivity
select = Select(title="Plot to show:", value="Line 1", options=["Line 1", "Line 2"])
select.js_on_change("value", CustomJS(args=dict(line_1=plot_1, line_2=plot_2), code="""

line_1.visible = true
line_2.visible = true

if (this.value === "Line 1") {
    line_2.visible = false 
} else {
    line_1.visible = false
}
    
"""))

layout = column(select, p)
show(layout)

Loading BokehJS ...

In [ ]:
def graph_request_handler(request):
    if request["method"]=="GET":
        #parse the body of the incoming POST
        try:
            username = request['values']['user']
            gamename = request['values']['game_name']
        except:
            return "Error with game or username"   
        
        
        with sqlite3.connect(game_db) as c:
            c.cursor()  # move cursor into database (allows us to execute commands)
            
            users = [i[0] for i in c.execute('''SELECT DISTINCT user FROM all_game_data;''')]
            print(users)
            scores = []
            for u in users:
                top_score = [i for i in c.execute('''SELECT * FROM all_game_data WHERE user == ? AND game_name == ? ORDER BY length ASC LIMIT 1;''',(u,gamename))][0][3]
                scores.append(top_score)

            df = pd.DataFrame({'User': users, 'Best Score': scores}).sort_values(by=['Best Score'])
            source = ColumnDataSource(df)
            columns = [TableColumn(field='User', title='User'),TableColumn(field='Best Score', title='Best Score')]
            myTable = DataTable(source=source, columns=columns)


            user_games = [(i[1], i[3]) for i in c.execute('''SELECT * FROM all_game_data WHERE user == ? AND game_name == ? ORDER BY date ASC;''',(username,gamename))]
            p1 = figure(title="Score over time", width=300, height=300, x_axis_type="datetime")
            time = [datetime.datetime.strptime(u[0],'%Y-%m-%d %H:%M:%S.%f') for u in user_games]
            score = [u[1] for u in user_games]
            print(time)
            print(score)
            p1.line(time, score)

            col = column(myTable, p1)
            show(col)
            script1, div1 = components(col)

        return f'''<!DOCTYPE html>
                <html> <script src="https://cdn.bokeh.org/bokeh/release/bokeh-2.4.0.min.js"></script>
                <body>
                    {div1}
                    {script1}
                </body>
                </html>
                '''
    else:
        return "invalid HTTP method for this url."

In [88]:
def graph_request_handler(request):
    if request["method"]=="GET":
        #parse the body of the incoming POST
        try:
            username = request['values']['user']
        except:
            return "Error with username"   
        
        
        with sqlite3.connect(game_db) as c:
            c.cursor()  # move cursor into database (allows us to execute commands)
            
            users = [i[0] for i in c.execute('''SELECT DISTINCT user FROM all_game_data;''')]
            print(users)
            games = [j[0] for j in c.execute('''SELECT DISTINCT game_name FROM all_game_data;''')]
            print(games)

            scores = []
            gn = games[0]
            for u in users:
                top_score = [i for i in c.execute('''SELECT * FROM all_game_data WHERE user == ? AND game_name == ? ORDER BY length ASC LIMIT 1;''',(u,gn))][0][3]
                scores.append(top_score)

            df = pd.DataFrame({'User': users, 'Best Score': scores}).sort_values(by=['Best Score'])
            source = ColumnDataSource(df)
            columns = [TableColumn(field='User', title='User'),TableColumn(field='Best Score', title='Best Score')]
            myTable = DataTable(source=source, columns=columns)


            user_games = [(i[1], i[3]) for i in c.execute('''SELECT * FROM all_game_data WHERE user == ? AND game_name == ? ORDER BY date ASC;''',(username,gamename))]
            p1 = figure(title="Score over time", width=300, height=300, x_axis_type="datetime")
            time = [datetime.datetime.strptime(u[0],'%Y-%m-%d %H:%M:%S.%f') for u in user_games]
            score = [u[1] for u in user_games]
            print(time)
            print(score)
            p1.line(time, score)

            col = column(myTable, p1)
            show(col)
            script1, div1 = components(col)

        return f'''<!DOCTYPE html>
                <html> <script src="https://cdn.bokeh.org/bokeh/release/bokeh-2.4.0.min.js"></script>
                <body>
                    {div1}
                    {script1}
                </body>
                </html>
                '''
    else:
        return "invalid HTTP method for this url."

In [58]:
user_game = {'method': 'POST', 'form': {'user': 'mat', 'game_name': 'maze', 'length': '14.5'}, 'args': ['user', 'game_name', 'length']}
request_handler(user_game)

'Data POSTED successfully'

In [111]:
ug = {'method': 'GET', 'values': {'user': 'cat'}, 'args': ['game_name']}
graph_request_handler(ug)

['cat']
['maze', 'math']


NameError: name 'gamename' is not defined

In [169]:
Func = open("test.html","w")
result = graph_request_handler(ug)
Func.write(result)
Func.close()

['cat', 'bat', 'mat']
[datetime.datetime(2022, 4, 13, 2, 21, 31, 733577), datetime.datetime(2022, 4, 20, 21, 7, 56, 16545), datetime.datetime(2022, 4, 20, 21, 9, 52, 952268)]
[12.5, 12.5, 13.5]


password + login


In [2]:
ht_db = 'database/userpassword.db' 

In [10]:
def create_login_request_handler(request):
    if request["method"]=="POST":
        #parse the body of the incoming POST
        try:
            username = request['form']['user']
            pw = request['form']['password']
        except:
            return "Error with username or password"   
        
        with sqlite3.connect(ht_db) as c:
            c.cursor()  # move cursor into database (allows us to execute commands)
            c.execute("""CREATE TABLE IF NOT EXISTS user_pw_data (user text, password text);""")
            # some other query(ies) about inserting data
            
            #if already in database
            users = [i for i in c.execute('''SELECT * FROM user_pw_data WHERE user == ?;''',(username,))]
            
            print('users ',users)
            if users != []:
                return "Username is taken"
            else:
                c.execute('''INSERT into user_pw_data VALUES (?,?);''',(username, pw))
        
        return "Data POSTED successfully"
    else:
        return "invalid HTTP method for this url."

In [4]:
r = {'method': 'POST', 'form': {'user': 'cat', 'password': '2006'}, 'args': ['user', 'password']}

In [11]:
create_login_request_handler(r)

users  [('cat', '2006')]


'Username is taken'

In [30]:
g = {'method': 'GET', 'values': {'user': 'cat', 'password': '2006'}, 'args': ['user', 'password']}

In [31]:
def login_request_handler(request):
    if request["method"]=="GET":
        #parse the body of the incoming POST
        try:
            username = request['values']['user']
            pw = request['values']['password']
        except:
            return "This request is missing a username or password"   
        
        with sqlite3.connect(ht_db) as c:
            
            #check if exists
            users = [i for i in c.execute('''SELECT * FROM user_pw_data WHERE user == ?;''',(username,))]
            
            if users:
                print(users[0][1])
                if users[0][1] != pw:
                    return "Incorrect password"

                return "Login complete"
            else:
                return "Incorrect username or user does not exist"
            
        return users
    else:
      return "invalid HTTP method for this url."

In [32]:
login_request_handler(g)

2006


'Login complete'

In [37]:
tester = {'method': 'POST', 'args': ['user', 'password'], 'form': {'user': 'kxzheng', 'password': '300'}}

In [38]:
create_login_request_handler(tester)

users  []


'Data POSTED successfully'

In [39]:
tester1 = {'method': 'GET', 'args': ['user', 'password'], 'values': {'user': 'kxzheng', 'password': '300'}}

In [40]:
login_request_handler(tester1)

300


'Login complete'